<!-- 파일 경로: C:/githome/9-11week_py_statistics/42_dbscan_corrected.ipynb -->

# DBSCAN: 밀도 기반 군집화 알고리즘 (수정본)

**학습 목표:**
1. K-Means와 다른, 밀도 기반의 군집화 알고리즘인 **DBSCAN**의 원리를 이해합니다.
2. DBSCAN을 사용하여 K-Means로는 군집화하기 어려운 복잡한 분포(예: 초승달 모양)의 데이터를 효과적으로 군집화합니다.
3. DBSCAN의 주요 하이퍼파라미터인 `eps`와 `min_samples`의 의미를 이해하고, 이를 조정하여 군집화 결과를 개선합니다.

## 1. DBSCAN (Density-Based Spatial Clustering of Applications with Noise) 이란?

DBSCAN은 데이터가 밀집된 정도, 즉 **밀도**를 기반으로 군집을 형성하는 알고리즘입니다. K-Means처럼 군집의 개수(k)를 미리 지정할 필요가 없으며, 기하학적으로 복잡한 모양의 데이터셋도 잘 찾아내는 장점이 있습니다.

### 주요 개념
- **`eps` (epsilon)**: 개별 데이터를 중심으로 하는 원(또는 구)의 반경입니다. 이 반경 안에 다른 데이터가 얼마나 있는지로 밀도를 측정합니다.
- **`min_samples`**: `eps` 반경 내에 최소한 이 개수 이상의 데이터가 있어야 밀집 지역으로 인정합니다.
- **핵심 포인트(Core Point)**: `eps` 반경 내에 `min_samples` 개수 이상의 데이터(자기 자신 포함)를 가진 데이터입니다.
- **경계 포인트(Border Point)**: `eps` 반경 내에 `min_samples`보다는 적은 데이터가 있지만, 핵심 포인트의 반경 내에는 속하는 데이터입니다.
- **잡음 포인트(Noise Point)**: 핵심 포인트도, 경계 포인트도 아닌 데이터입니다. DBSCAN은 이를 어떤 군집에도 속하지 않는 **이상치(Outlier)**로 취급합니다. (레이블: -1)

In [ ]:
# C:/githome/9-11week_py_statistics/42_dbscan_corrected.ipynb

# 필요한 라이브러리를 임포트합니다.
from sklearn.cluster import DBSCAN       # DBSCAN 알고리즘
from sklearn.datasets import make_moons  # 초승달 모양의 가상 데이터 생성
import matplotlib.pyplot as plt          # 시각화 라이브러리
import numpy as np
import pandas as pd

### 예제 1: 초승달 모양 데이터 군집화

In [ ]:
# K-Means로는 군집화하기 어려운 초승달 모양의 데이터를 생성합니다.
X, _ = make_moons(n_samples=300, noise=0.05, random_state=42)

In [ ]:
# DBSCAN 모델을 생성하고 학습시킵니다.
dbscan = DBSCAN(eps=0.2, min_samples=5)
dbscan.fit(X)

In [ ]:
# 각 데이터 포인트에 할당된 군집 레이블을 확인합니다.
dbscan.labels_

In [ ]:
# 군집화 결과를 시각화합니다.
plt.scatter(X[:,0], X[:,1], c=dbscan.labels_, cmap='rainbow')
plt.show()

## 2. 예제 2: 붓꽃(Iris) 데이터 군집화

In [ ]:
# 붓꽃 데이터셋을 로드하고 DataFrame으로 변환합니다.
from sklearn.datasets import load_iris
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
iris_df['target'] = iris.target

In [ ]:
# 붓꽃 데이터에 DBSCAN을 적용합니다.
iris_dbscan = DBSCAN(eps=0.6, min_samples=8, metric='euclidean')
dbscan_labels = iris_dbscan.fit_predict(iris.data)

# 군집화 결과를 DataFrame에 추가합니다.
iris_df['dbscan_cluster'] = dbscan_labels
iris_df.groupby(['target'])['dbscan_cluster'].value_counts()

### 붓꽃 군집화 결과 시각화 (PCA 사용)
4차원 데이터를 2차원으로 축소하여 시각화합니다.

In [ ]:
# 시각화를 위한 함수 정의 (원본 노트북의 함수와 동일)
def visualize_cluster_plot(clusterobj, dataframe, label_name, iscenter=True):
    unique_labels = np.unique(dataframe[label_name].values)
    markers = ['o', 's', '^', 'x', '*', 'P', 'D', 'v']
    isNoise = False

    if iscenter and hasattr(clusterobj, 'cluster_centers_'):
        centers = clusterobj.cluster_centers_
    else:
        iscenter = False

    for label in unique_labels:
        label_cluster = dataframe[dataframe[label_name] == label]
        if label == -1:
            cluster_legend = 'Noise'
            isNoise = True
        else:
            cluster_legend = f'Cluster {label}'

        marker_style = markers[label % len(markers)] if label != -1 else 'X'

        plt.scatter(x=label_cluster['ftr1'], y=label_cluster['ftr2'],
                    s=70, edgecolor='k', marker=marker_style, label=cluster_legend)

        if iscenter and label != -1:
            center_x_y = centers[label]
            plt.scatter(x=center_x_y[0], y=center_x_y[1], s=250, color='white',
                        edgecolor='k', alpha=0.9, marker=marker_style)
            plt.scatter(x=center_x_y[0], y=center_x_y[1], s=70, color='k',
                        edgecolor='k', marker=f'${label}$')

    plt.legend(loc='upper center' if isNoise else 'upper right')
    plt.title(f"Clustering result: {label_name}")
    plt.xlabel("ftr1 (PCA 1)")
    plt.ylabel("ftr2 (PCA 2)")
    plt.grid(True)
    plt.show()

In [ ]:
# PCA를 사용하여 붓꽃 데이터를 2차원으로 축소합니다.
from sklearn.decomposition import PCA
pca = PCA(n_components=2, random_state=0)
pca_transformed = pca.fit_transform(iris.data)
iris_df['ftr1'] = pca_transformed[:, 0]
iris_df['ftr2'] = pca_transformed[:, 1]

In [ ]:
# DBSCAN 군집화 결과를 시각화합니다.
visualize_cluster_plot(iris_dbscan, iris_df, 'dbscan_cluster', iscenter=False)

### 하이퍼파라미터 튜닝

`eps` 값을 0.6에서 0.5로 줄여서 잡음점을 줄여보겠습니다. `eps`가 작아지면 더 촘촘한 지역만 군집으로 인정하게 됩니다.

In [ ]:
# eps 값을 0.5로 변경하여 다시 군집화를 수행합니다.
iris_dbscan_tuned = DBSCAN(eps=0.5, min_samples=8, metric='euclidean')

# [중요] fit_predict는 원본 데이터(iris.data)로 수행해야 합니다.
# PCA로 변환된 데이터는 시각화를 위한 것일 뿐, 군집화는 원본 데이터의 밀도를 기반으로 해야 합니다.
dbscan_labels_tuned = iris_dbscan_tuned.fit_predict(iris.data)
iris_df['dbscan_cluster_tuned'] = dbscan_labels_tuned

# 튜닝된 군집화 결과를 2D PCA 데이터로 시각화합니다.
visualize_cluster_plot(iris_dbscan_tuned, iris_df, 'dbscan_cluster_tuned', iscenter=False)

## 정리

- **DBSCAN**은 밀도 기반 군집화로, K-Means와 달리 **군집 개수를 미리 지정할 필요가 없고** 복잡한 형태의 군집도 잘 찾아냅니다.
- **`eps`**와 **`min_samples`**가 중요한 하이퍼파라미터이며, 이 값들을 어떻게 설정하느냐에 따라 군집 결과와 **잡음점(Noise)**의 개수가 달라집니다.
- DBSCAN은 데이터의 밀도가 균일하지 않을 때 성능이 떨어질 수 있으며, `eps`와 `min_samples`를 데이터에 맞게 잘 설정하는 것이 중요합니다.